In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("preprocessedNP.csv")
# print(data.head())
data.columns = ['text', 'label']

In [2]:
data.head()

,text,label
0,Thank you happy.1,0
1,That would be a great trick happy.3,0
2,unhappy every time laughing my ass off,1
3,have a blast okay. love you!,1
4,Enjoy happy,0


In [3]:
# print(data.isnull().sum())

In [4]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["text"] = data["text"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sourabh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data["label"] = data["label"].map({0: "No Stress", 1: "Stress"})
data = data[["text", "label"]]
print(data.head())

                            text      label
0                         thank   No Stress
1             would great trick   No Stress
2  unhappi  everi time laugh ass     Stress
3                blast okay love     Stress
4                    enjoy happi  No Stress


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

x = np.array(data["text"])
y = np.array(data["label"])

cv = CountVectorizer()
X = cv.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X, y,test_size=0.33,random_state=42)

In [7]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
model = BernoulliNB()
model.fit(xtrain, ytrain)
# print("Accuracy of Logistic Regression model is:",
# metrics.accuracy_score(ytest, ytrain)*100)

BernoulliNB()

In [30]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = model.predict(data)
print(output)

['No Stress']


In [29]:
model.score(xtest,ytest)*100

90.26315789473685

In [28]:
from sklearn.metrics import classification_report
z = model.predict(xtest)
print(classification_report(ytest, z))

              precision    recall  f1-score   support

   No Stress       0.91      0.91      0.91       423
      Stress       0.89      0.89      0.89       337

    accuracy                           0.90       760
   macro avg       0.90      0.90      0.90       760
weighted avg       0.90      0.90      0.90       760

